<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/productionOfHydrogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Production of hydrogen
#@markdown This page will give an introduction to production of hydrogen.
#@markdown <br><br>This document is part of the module ["Introduction to Gas Processing using NeqSim in Colab"](https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/examples_of_NeqSim_in_Colab.ipynb#scrollTo=_eRtkQnHpL70).
%%capture
!pip install neqsim==2.5.35
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
plt.style.use('classic')
%matplotlib inline

In [1]:
%%capture
#https://reaktoro.org/installation/installation-in-google-colab.html
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

#Litterature

https://www.equinor.com/en/what-we-do/hydrogen.html



# Hydrogen production

## Steam methane reforming
https://en.wikipedia.org/wiki/Steam_reforming

Steam reforming or steam methane reforming (SMR) is a method for producing syngas (hydrogen and carbon monoxide) by reaction of hydrocarbons with water. Commonly natural gas is the feedstock. The main purpose of this technology is hydrogen production.

## 1. pre-reforming

The purpose of pre-reforming is to break down higher hydrocarbons such as propane, butane or naphtha into methane (CH4), which allows for more efficient reforming downstream.

In [15]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 C2H6 C3H8 CO CO2 H2O")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(1200, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("C2H6",  0.05, "mol")
state.set("C3H8",  0.01, "mol")
state.set("H2O",  0.1, "mol")

equilibrate(state)

print(state)

+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1473.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.5800e+00 |  mol |
| :: C            | 1.1300e+00 |  mol |
| :: O            | 1.0000e-01 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.1053e+00 |  mol |
| :: C2H6         | 8.3564e-04 |  mol |
| :: C3H8         | 1.8350e-06 |  mol |
| :: CO           | 2.2840e-02 |  mol |
| :: CO2          | 1.5997e-04 |  mol |
| :: H2O          | 7.6840e-02 |  mol |
+-----------------+------------+------+


In [16]:
solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1473.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.5800e+00 |  mol |
| :: C            | 1.1300e+00 |  mol |
| :: O            | 1.0000e-01 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.1053e+00 |  mol |
| :: C2H6         | 8.3564e-04 |  mol |
| :: C3H8         | 1.8350e-06 |  mol |
| :: CO           | 2.2840e-02 |  mol |
| :: CO2          | 1.5997e-04 |  mol |
| :: H2O          | 7.6840e-02 |  mol |
+-----------------+------------+------+


## 2. Steam reforming

The name-giving reaction is the steam reforming (SR) reaction and is expressed by the equation:

In [2]:
from reaktoro import *
import numpy as np
import pandas as pd
import math as math

db = NasaDatabase("nasa-cea")
db = NasaDatabase.withName("nasa-cea")

#print(f"{'Species':<20}{'Formula':<60}{'Molar Mass (kg/mol)':<20}")
#for species in db.species():
#    print(f"{species.name():<20}{species.formula().str():<60}{species.molarMass():<20.6f}")

rxn = db.reaction("CO + 2*H2 = CH3OH")

rprops = rxn.props(25.0, "C", 1.0, "atm")
print(rprops)

#db = SupcrtDatabase("supcrtbl")
#db = SupcrtDatabase.withName("supcrtbl")

#gases = GaseousPhase("N2 CH4 C2H6 O2 CO2 CO H2O H2 NO NO2")
gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")
gases.set(ActivityModelPengRobinson())

solution = CondensedPhase("H2O(l) CH3OH(l)")

system = ChemicalSystem(db, gases,solution)

+------------------------------------------------+-------------+-------------+
| Property                                       |       Value |        Unit |
+------------------------------------------------+-------------+-------------+
| Temperature                                    |    298.1500 |           K |
| Pressure                                       |      1.0132 |         bar |
| Equilibrium Constant (log base 10)             |      4.3880 |           - |
| Delta Standard Gibbs Energy                    | -25046.6852 |       J/mol |
| Delta Standard Enthalpy                        | -90404.2836 |       J/mol |
| Delta Standard Volume                          |  0.0000e+00 |      m3/mol |
| Delta Standard Volume (Temperature Derivative) |  0.0000e+00 |  m3/(mol*K) |
| Delta Standard Volume (Pressure Derivative)    |  0.0000e+00 | m3/(mol*Pa) |
| Delta Standard Isobaric Heat Capacity          |    -42.7743 |   J/(mol*K) |
| Delta Standard Isochoric Heat Capacity         |  

In [3]:
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(1000, "bar")
state.set("CH4", 1.0, "mol")
#state.set("C2H6", 0.04, "mol")
state.set("O2",  1.5, "mol")
#state.set("N2",  0.5, "mol")

print("=== INITIAL STATE ===")
print(state)

=== INITIAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |  1273.1500 |    K |
| Pressure            |  1000.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 4.0000e+00 |  mol |
| :: C                | 1.0000e+00 |  mol |
| :: O                | 3.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 1.0000e+00 |  mol |
| :: O2               | 1.5000e+00 |  mol |
| :: CO2              | 1.0000e-16 |  mol |
| :: CO               | 1.0000e-16 |  mol |
| :: H2O              | 1.0000e-16 |  mol |
| :: H2               | 1.0000e-16 |  mol |
| :: H2O(l)           | 1.0000e-16 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+


In [4]:
solver = EquilibriumSolver(system)
solver.solve(state)  # equilibrate the `state` object!

print("=== FINAL STATE ===")
print(state)

=== FINAL STATE ===
+---------------------+------------+------+
| Property            |      Value | Unit |
+---------------------+------------+------+
| Temperature         |  1273.1500 |    K |
| Pressure            |  1000.0000 |  bar |
| Charge:             | 0.0000e+00 |  mol |
| Element Amount:     |            |      |
| :: H                | 4.0000e+00 |  mol |
| :: C                | 1.0000e+00 |  mol |
| :: O                | 3.0000e+00 |  mol |
| Species Amount:     |            |      |
| :: CH4              | 1.1322e-01 |  mol |
| :: O2               | 1.1275e-16 |  mol |
| :: CO2              | 6.4407e-01 |  mol |
| :: CO               | 2.4270e-01 |  mol |
| :: H2O              | 1.4691e+00 |  mol |
| :: H2               | 3.0441e-01 |  mol |
| :: H2O(l)           | 1.0000e-16 |  mol |
| :: CH3OH(l) :: CH4O | 1.0000e-16 |  mol |
+---------------------+------------+------+


In [5]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

equilibrate(state)

print(state)

+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        |   100.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 3.7606e-01 |  mol |
| :: O2           | 1.0000e-16 |  mol |
| :: CO2          | 9.3630e-02 |  mol |
| :: CO           | 5.3031e-01 |  mol |
| :: H2O          | 2.8243e-01 |  mol |
| :: H2           | 9.6544e-01 |  mol |
+-----------------+------------+------+
